_____

<table align="left" width=100%>
    <td>
        <div style="text-align: center;">
          <img src="./images/bar.png" alt="entidades financiadoras"/>
        </div>
    </td>
    <td>
        <p style="text-align: center; font-size:24px;"><b>Introduction to Data Science</b></p>
        <p style="text-align: center; font-size:18px;"><b>Master in Electrical and Computer Engineering</b></p>
        <p style="text-align: center; font-size:14px;"><b>Pedro Cardoso (pcardoso@ualg.pt)</b></p>
    </td>
</table>

_____

# Exploratory Data Analysis with Pandas (part 3)

Let us continue with the exploratory data analysis with Pandas. We will continue with the titanic dataset. So, load the titanic dataset and remember its structure.

In [ ]:
# load necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_bokeh

pandas_bokeh.output_notebook()
%matplotlib inline

Load the titanic dataset.

In [ ]:
df_titanic = pd.read_excel('./data/titanic/Titanic.xls')
df_titanic.head()

## Data types in Pandas

Pandas has a `dtypes` attribute that returns the data types of the columns in the data frame. Possible data types include
- `int64`, 
- `float64`, 
- `object`, 
- `bool`, 
- `datetime64`, 
- `timedelta[ns]`, and 
- `category`. 

By default, the data types are inferred from the data. If the data type is numeric, numpy data types are used. If the data type is non-numeric, the data type is inferred as `object`. The `object` data type is used for string values and for other mixed data types. (see https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes for more details)

For example, let us check the data types of the titanic dataset using the `dtypes` attribute. 

In [ ]:
df_titanic.dtypes

Which can be also visualized as follows:

In [ ]:
df_titanic.info()

We can get the data types of a specific column using the `dtype` attribute.

In [ ]:
df_titanic['survived'].dtype

And get all columns of a specific data type using the `select_dtypes` method.

Such as all columns of type `int64`.

In [ ]:
df_titanic.select_dtypes(include=['int64'])

Or all columns of type `float64`.

As a note, `body` is considered as a float because it contains missing values, i.e., `NaN` (because `NaN` is a float, this forces an array of integers with any missing values to become floating point). 

In [ ]:
df_titanic.select_dtypes(include=['float64'])

Or all columns of type `object`.
Object data type is used for string values and for other mixed data types.

In [ ]:
df_titanic.select_dtypes(include=['object'])

Or, all numeric columns.

In [ ]:
df_titanic.select_dtypes(include=['number'])

The conversion of the data types can be done using the `astype` method. For example, let us convert the `survived` feature (containing 0 and 1) to `bool`.

In [ ]:
# copy the data frame
df = df_titanic.copy()

# convert the survived feature to bool
df['survived'] = df['survived'].astype(bool)

# check the data types
df.dtypes

Looking at the results, we can see that the `survived` feature is now of type `bool`, i.e., `True` and `False`.

In [ ]:
df['survived']

Other transformations are natural such as converting the `embarked` and `sex` feature to `category`, as follows:

In [ ]:
df['embarked_category'] = df['embarked'].astype('category')
df['sex_category'] = df['sex'].astype('category')
df.dtypes

This does not change the values of the feature, but it changes the data type, the `pclass` feature is now of type `category`.

In [ ]:
df[['embarked_category', 'sex_category']]

The `pcalss` feature can also be considered as a categorical feature. Let us convert it to `category` data type.

In [ ]:
df['pclass_category'] = df['pclass'].astype('category')

### Performance of the `category` data type vs. `object` data type (optional)

This is useful for performance reasons, as in some cases, the `category` data type is more efficient than `object` data type. For example, let us compare the performance of the `category` and `object` data types when counting the number of unique values.

In [ ]:
def timeit_cat_vs_other(df, feature):
    print()
    feature_category = f'{feature}_category'
    print(f'Time to count the number of unique values in the {feature} feature (dtype = {df[feature].dtype}) vs. {df[feature_category].dtype})')
    %timeit df[feature].nunique()
    %timeit df[feature_category].nunique()

timeit_cat_vs_other(df, 'sex')
timeit_cat_vs_other(df, 'embarked')

Also in terms of memory usage, the `category` data type is more efficient than `object` data type.

In [ ]:
df[['embarked', 'embarked_category', 'sex', 'sex_category', 'pclass', 'pclass_category']].memory_usage(deep=True)

Transformations of categorical columns can also have significant impact on the performance of some methods.

In [ ]:
print('Time to convert the sex feature to upper case')
%timeit df['sex'].str.upper()
%timeit df['sex_category'].str.upper()

print()
print('Time to convert the embarked feature to upper case')
%timeit df['embarked'].str.upper()
%timeit df['embarked_category'].str.upper()

For other operations the `category` data type is **not** more efficient than `object` data type. For example, let us compare the performance of the `category`, `object` and `int64` a data types when counting the number of occurrences of each value. 

This is because the `category` data type is stored as an array of integers, and the `object` data type is stored as an array of pointers to the strings. 

In [ ]:
print('Time to count the number of occurrences of each value in the embarked feature')
%timeit df['embarked'].value_counts()
%timeit df['embarked_category'].value_counts()

print()
print('Time to count the number of occurrences of each value in the pclass feature')
%timeit df['pclass'].value_counts()
%timeit df['pclass_category'].value_counts()

Or when doing a group by operation.

In [ ]:
print('Time to group by the embarked feature')
%timeit df.groupby('embarked').size()
%timeit df.groupby('embarked_category').size()

In [ ]:
print('Time to group by pclass feature')
%timeit df.groupby('pclass').size()
%timeit df.groupby('pclass_category').size()

So, the conversion of the data types can have significant impact on the performance. In general, the `category` data type is more efficient than `object` data type, but not always. This must be evaluated on a case by case basis.

## Feature encoding
Feature encoding is the process of transforming the features to have a more machine learning friendly format. For example, categorical features are transformed to have integer values. This is useful for some machine learning algorithms.

### One-hot encoding

One-hot encoding is the process of transforming the features to have a more machine learning friendly format. For example, categorical features are split into multiple binary features.

In [ ]:
df = df_titanic.copy()

encoded_data = pd.get_dummies(df[['embarked', 'sex']],
                              dtype=int # default is bool
)

encoded_data

In [ ]:
# add the encoded data to the original data frame and drop the original features
df = pd.concat([df, encoded_data], axis=1).drop(['embarked', 'sex'], axis=1)
df

### Label encoding
Label encoding is the process of transforming the features to, usually, have a more machine learning friendly format. For example, categorical features are transformed to have integer values.

Let us encode the `embarked` feature.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# copy the data frame
df = df_titanic.copy()

# encode the embarked feature
le = LabelEncoder()
df['embarked'] = le.fit_transform(df['embarked'])

df

The `LabelEncoder` class has a `classes_` attribute that contains the list of classes that were encoded.

In [ ]:
le.classes_

As an alternative to `LabelEncoder`, for categorial features we can use the `cat.codes` accessor.

In [ ]:
df = df_titanic.copy()
df['embarked'] = df['embarked'].astype('category')
df['embarked'] = df['embarked'].cat.codes
df['embarked']

### Ordinal encoding

Ordinal encoding is similar to label encoding, but the classes are ordered. For example, the classes `low`, `medium`, and `high` can be encoded as `0`, `1`, and `2`, respectively.

In the Titanic dataset, we can consider the `pclass` feature as an ordinal feature.   

In [ ]:
df_titanic.pclass

## Feature scaling

In practical scenarios, features often have distinct ranges, magnitudes, and units. For instance, age may vary between 0 and 120, while salary can fluctuate between zero and thousands or even millions. This raises the question of how data analysts or scientists can compare such features, given that they are on different scales. It is worth noting that high-magnitude features tend to have a more significant impact on machine learning models than lower magnitude ones. Fortunately, feature scaling or normalization can help address these issues.

So, feature scaling refers to the process of bringing all features to the same magnitude level. It is not mandatory for all algorithms, but some algorithms necessitate scaled data, such as those that depend on Euclidean distance measures, like K-nearest neighbor and K-means clustering algorithms.

Scalllin can also be used to annoniymize the data. For example, the age of a person can be scaled to the range [0, 1] by dividing by 100. This way, the age of a person is not directly available in the data.

### Standard scaling
Standard scaling is the process of transforming the features to have a more normal distribution. The standard scaling is performed by subtracting the mean and dividing by the standard deviation., i.e., $$x_{scaled} = \frac{x - \mu}{\sigma}$$

In [ ]:
from sklearn.preprocessing import StandardScaler

# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# standard scaling
scaler = StandardScaler()
scaler.fit(df[features])


df[features] = scaler.transform(df[features])

df

Let us see the distribution of the original data and the scaled data.


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

df_titanic[['age', 'fare']].plot.hist(alpha=0.5, bins=10, title='original data', ax=ax[0])
df[['age', 'fare']].plot.hist(alpha=0.5, bins=10, title='standard scaled data', ax=ax[1])

The distribution of the original data and the scaled data is different. The scaled data has a mean of zero and a standard deviation of one. The scaled data is centered around zero, and the values are almost all within the range of -3 and 3. The scaled data is more normally distributed than the original data.

However, plotting the data in a scatter plot, we can see that the shape of the data is the same, only the scale is different.

In [ ]:
ax = df_titanic['age'].sort_values().reset_index().drop('index', axis=1).plot(style='o', title='original data')
df['age'].sort_values().reset_index().drop('index', axis=1).plot(style=".",ax=ax, secondary_y=True)


### Min-max scaling

Min-max scaling is the process of transforming the features to have a more uniform distribution. The min-max scaling to the [0, 1] interval is performed by subtracting the minimum and dividing by the range., i.e., $$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}.$$

To transform the data to the $[a, b]$ interval, we can use the following formula: $$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}  (b - a) + a.$$

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# min-max scaling
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

df

Let us see the distribution of the original data and the scaled data.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

df_titanic[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='original data', ax=ax[0])
df[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='min-max scaled data', ax=ax[1])

The distribution of the original data and the scaled data is different. The scaled data is in the [0, 1] interval. The scaled data is more uniformly distributed than the original data.

### Robust scaling
Robust scaling features in a way that accounts for outliers. The method achieves this by first removing the median and then scaling the data based on the quantile range. The default quantile range used is the Interquartile Range (IQR), although it can be customized if needed.

During the scaling process, each feature is centered and scaled independently by computing relevant statistics from the training set. Outliers can often skew the sample mean and variance in undesirable ways.

So, robust scaling is computed as follows:
 $$x_{scaled} = \frac{x - \text{median}(x)}{\text{IQR}(x)},$$
where $\text{median}(x)$ is the median of the feature $x$, and $\text{IQR}(x)$ is the interquartile range of the feature $x$.

In [ ]:
from sklearn.preprocessing import RobustScaler

# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# robust scaling
scaler = RobustScaler()
df[features] = scaler.fit_transform(df[features])

df

Let us see the distribution of the original data and the scaled data. 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

df_titanic[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='original data', ax=ax[0])
df[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='robust scaled data', ax=ax[1])

The distribution of the original data and the scaled data is different. The scaled data is centered around zero and is more uniformly distributed than the original data. The scaled data is more robust to outliers than the original data.


## Feature transformation
Feature transformation is the process of transforming the features allows, for exameple, reducing the skewness of the features, the effect of outliers, etc. Examples feature transformations are log transformation, square root transformation, square, etc.

If the feature is right-skewed or positively skewed or grouped at lower values, then we can apply the square root, cube root, and logarithmic transformations, while if the feature is left-skewed or negative skewed or grouped at higher values, then we can apply the cube, square, and so on.

### Log transformation

The log transformation is performed by taking the logarithm of the feature., i.e., $$x_{log} = \log(x).$$

In [ ]:
from scipy import stats

# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# log transformation
df[features] = df[features].apply(lambda x: np.log(x + 1))

df

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

df_titanic[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='original data', ax=ax[0])
df[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='log scaled data', ax=ax[1])

### Square root transformation
Square root transformation is performed by taking the square root of the feature., i.e., $$x_{sqrt} = \sqrt{x}.$$

In [ ]:
# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# square transformation
df[features] = df[features].apply(lambda x: np.sqrt(x))

df

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

df_titanic[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='original data', ax=ax[0])
df[['age', 'fare']].plot.hist(alpha=0.5, bins=20, title='log scaled data', ax=ax[1])

## Discretization transformation
Discretization transformation is performed by transforming numerical features to categorical features. This is useful for some machine learning algorithms. For example, the following code transforms the age feature to a categorical feature.

To do this, we use the `pd.cut` function. The `pd.cut` function takes as input the feature to be transformed, the bins, and the labels. The bins are the intervals in which the feature will be transformed. The labels are the names of the categories.

In [ ]:
# copy the data frame
df = df_titanic.copy()

# select the features
features = ['age', 'fare']

# discretization transformation
df['age category'] = pd.cut(df['age'], bins=[0, 18, 30, 65, 100], labels=['child', 'young', 'adult', 'senior'])

df

In [ ]:
df[['age', 'age category']].groupby('age category', observed=True).count().plot(kind='bar')

## Feature splitting
Feature splitting is the process of splitting a feature into multiple features. For example, some times is possible to split the `name` feature into two features: `first name` and `last name`. Or the spliting of a `date` feature into three features: `year`, `month`, and `day`.


### Dates example

Consider the dataset with energy consumption of a house. The dataset contains the date and the energy consumption of the house. The date feature can be split into multiple features, such as year, month, day, hour, minute, and second. This can be done using the `dt` accessor.

In [ ]:
df_consumption = pd.read_csv('./data/house_consumption_TS/house_consumption.csv')
df_consumption.head()

In [ ]:
df_consumption.info()

In [ ]:
df_consumption['date'] = pd.to_datetime(df_consumption['date'])
df_consumption.info()

In [ ]:
df_consumption['year'] = df_consumption['date'].dt.year
df_consumption['month'] = df_consumption['date'].dt.month
df_consumption['day'] = df_consumption['date'].dt.day
df_consumption['hour'] = df_consumption['date'].dt.hour
df_consumption['minute'] = df_consumption['date'].dt.minute
df_consumption['second'] = df_consumption['date'].dt.second
df_consumption

### Titanic example

In the Titanic dataset, we can split the cabin feature into two features: `cabin number` and `cabin letter`. The latter corresponds to the deck of the Titanic. Furhter, some passenger have more than one cabin. In this case, we can split the cabin feature into multiple features, one for each cabin.

In [ ]:
df_titanic

The `extract` function can be used to extract the deck from the cabin feature. The `extract` function takes as input a regular expression that defines the pattern to be extracted. For example, the following code extracts the deck from the cabin feature.

In [ ]:
# copy the data frame
df = df_titanic.copy()

# select the features
features = ['cabin']

# feature splitting
df['cabin number'] = df['cabin'].str.extract('([0-9]+)')
df['deck'] = df['cabin'].str.extract('([A-Z])')
df['number of cabins'] = df['cabin'].str.split().str.len()

df.head(15)

And now it can be interesting to see the distribution of the passenger class in different decks. This can be done using the `groupby` function and the `unstack` function.

In [ ]:
df[['pclass', 'deck']]\
    .groupby('deck').value_counts()\
    .unstack()

In [ ]:
df[['pclass', 'deck']] \
    .groupby('deck').value_counts() \
    .unstack()\
    .plot(kind="bar")

Or we can see the distribution of the passenger class in different decks using the `pivot_table` function.

In [ ]:
df.pivot_table(index='deck', columns='pclass', aggfunc='size', fill_value=0).plot(kind="bar")

Another thing we can check is the correlation between the number of cabins and the passenger class or the fare. We can see that the number of cabins is highly correlated fare but not with the passenger class.

In [ ]:
df[['number of cabins', 'pclass', 'fare']].corr()

We can also see the distribution of the mean fare in different passenger classes and number of cabins.

In [ ]:
df.pivot_table(index='pclass', columns='number of cabins', values='fare', aggfunc='size', fill_value=0)#.plot.bar()

### Regular expression in a nutshell

Regular expressions are a powerful tool for string manipulation. They are used to find, replace, and split strings. They are also used to extract information from strings. For example, we can extract the title of the passenger from the name feature (as we'll see later).

So, a regular expression (regex) is a pattern used to match character combinations in strings. Here's a nutshell tutorial on regular expressions:
- **Matching characters**: You can match specific characters by simply including them in the pattern. For example, the pattern "cat" matches the characters "c", "a", and "t".

- **Character classes**: You can match a set of characters by using character classes. For example, the pattern [abc] matches any of the characters "a", "b", or "c".

- **Metacharacters**: Certain characters have a special meaning in regex patterns. For example, the dot "." matches any character except a newline, and the asterisk "*" matches zero or more occurrences of the previous character.

- **Anchors**: Anchors are used to match the position of a string. The caret "^" matches the start of a string, and the dollar sign "\$" matches the end of a string.

- **Quantifiers**: Quantifiers are used to specify how many times a character or group of characters should appear. For example, the pattern a{3} matches exactly three occurrences of the letter "a", and the pattern a{3,5} matches three to five occurrences of the letter "a".

- **Alternation**: Alternation is used to match one of several possible patterns. For example, the pattern cat|dog matches either "cat" or "dog".

- **Grouping**: Grouping is used to group parts of a pattern together. This is useful for applying quantifiers or alternation to a group of characters. For example, the pattern (ab)+ matches one or more occurrences of the string "ab".

- "\d" is a metacharacter that represents any digit from 0 to 9. It is commonly used to match numbers in text, and it is equivalent to the character class [0-9].

- [a-z] is a character class that matches any lowercase letter from "a" to "z". It represents a range of characters between "a" and "z" inclusive. For example, the pattern c[a-z]t matches any three-letter word that starts with "c" and ends with "t", where the middle letter can be any lowercase letter. This would match words such as "cat", "cet", "cxt", and so on.

- Some more examples: 
    -The pattern [a-z]+ matches one or more consecutive lowercase letters;

    - The pattern [a-z]{3} matches exactly three consecutive lowercase letters;

    - The pattern ^[a-z]+$ matches a string that consists entirely of lowercase letters;

    - You can also use other character classes in regular expressions to match other types of characters, such as uppercase letters ([A-Z]), digits (\d), whitespace (\s), or non-word characters (\W).

See https://docs.python.org/3/library/re.html for more information on the Python regular expression module. See also (Campesato, 2018).

Test your regular expressions using https://regex101.com/ or https://regexr.com/

Let me give you an example.

In [ ]:
import re
name = 'Braund, Mr. Owen Harris & Bradley, Mrs. Anna Michaela'

def print_regex_result(explanation, name, regex):
    print(re.findall(regex, name), ":", explanation)

print_regex_result('title, extracted considering the dot ', name, '([A-Za-z]+)\.')
print_regex_result('title, extracted considering the dot ', name, '(\w+)\.')
print_regex_result('title, extracted considering the comma', name, ',\s([A-Za-z]+)')
print_regex_result('last name, extracted considering the comma', name, '([A-Za-z]+),')
print_regex_result( 'first name, extracted considering the dot', name, '\.\s([A-Za-zç]+)')
print_regex_result('first name and posterior name, extracted considering the space between words', name, '([A-Za-zç]+)\s([A-Za-z]+)')

print_regex_result('list of words', name, '([A-Za-z]+)')
print_regex_result('list of words', name, '(\w+)')
print_regex_result('list of names', name, '([A-Za-z]+),\s\w+.\s([A-Za-z]+)\s([A-Za-z]+)')

## Feature engineering
Feature engineering includes
- the creation of new features from the existing features, 
- the selection of features, 
- the extraction of features, 
- the reduction of features, and 
- the aggregation of features. 
                
We already saw how to create new features from the existing features, by extracting the deck from the cabin feature.

As another example, we can create a new columns with the title and last name of the passenger.

In [ ]:
# copy the data frame
df = df_titanic.copy()

# select the features
features = ['name']

# feature engineering
df['title'] = df['name'].str.extract('([A-Za-z]+)\.')
df['last name'] = df['name'].str.extract('([A-Za-z]+),')

df

## Feature selection

### Correlation as a feature selection technique
In feature selection we select the features that are useful for the analysis. We can select the features using the correlation with the target variable. For example, we can select the features that have a correlation with the target variable greater than 0.1 to predict the survival of the passenger.

In [ ]:
# select the numerical features
df = df_titanic.select_dtypes(include=np.number)

# compute the correlation with the target variable
df.corr()['survived'].abs().sort_values(ascending=False)

### Feature selection using the variance
We can also select the features using the variance. For example, we can select the features that have a variance greater than 0.1 to predict the survival of the passenger.

In [ ]:
# select the numerical features
df = df_titanic.select_dtypes(include=np.number)

# compute the variance
df.var().sort_values(ascending=False)

# select the features with variance greater than 0.1
df = df.loc[:, df.var() > 0.1]

df

# References

- Campesato, O. (2018). Regular expressions: Pocket primer. Mercury Learning and Information.
- https://www.kaggle.com/learn/pandas
- Navlani, A.,  Fandango, A.,  Idris, I. (2021). Python Data Analysis: Perform data collection, data processing, wrangling, visualization, and model building using Python. Packt. 3rd Edition
- Brandt. S. (2014). Data Analysis: Statistical and Computational Methods for Scientists and Engineers. Springer. 4th Edition
- https://eugenelohh.medium.com/data-analysis-on-the-titanic-dataset-using-python-7593633135f2